- In this we will see everything related to `Filed data types`

### Common Data-type

- Binry datatype

![Binary datatypes](../img-src/Storing_image_as_binry.png)

In [1]:
import base64

In [6]:
from elasticsearch import Elasticsearch
import json
from pprint import pprint

-   create an index

In [8]:
es= Elasticsearch(hosts="http://localhost:9200/")
pprint(es.info().body)

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tGYnSe7ISE2SFo0pu1RcHg',
 'name': '5d9dbd669d35',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


In [9]:
es.indices.create(index="test_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

- `test_index` created, Now read and image convert into base64 bitecode and then decode It in utf-8 string for storage

In [10]:
image_str_data=""
with open('../img-src/test_image.png', "rb") as imageFD:
    image_binary_data=imageFD.read()
    image_str_data= base64.b64encode(image_binary_data).decode("utf-8")



In [11]:
print(image_str_data[:100])

iVBORw0KGgoAAAANSUhEUgAAAyYAAAGeCAYAAABhMRUDAAAABHNCSVQICAgIfAhkiAAAABl0RVh0U29mdHdhcmUAZ25vbWUtc2Ny


-  insert the image str in test_index

In [12]:
# convert the image data in a json format
doc={
    "image_data": image_str_data
}

# store the doc in test_index

res=es.index(index="test_index", document=doc)



In [13]:
pprint(res.body)

{'_id': 'y980EpUBZI6kT9oaQFII',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [15]:
# see the automatic mapping
infered_mapping=es.indices.get_mapping(index="test_index")["test_index"]["mappings"]

In [16]:
pprint(infered_mapping)

{'properties': {'image_data': {'fields': {'keyword': {'ignore_above': 256,
                                                      'type': 'keyword'}},
                               'type': 'text'}}}


## JSON object storage

# 3.1 Text

In [17]:
es.indices.delete(index="test_index", ignore_unavailable= True)
mapping={
    "properties":{
        "email_body":{"type":"text"}
    }
}
es.indices.create(index="test_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [18]:
doc={
    "email_body":"Hello, this is a test email body. regards, verma"
}
res=es.index(index="test_index", document=doc)
pprint(res.body)

{'_id': 'zN9VEpUBZI6kT9oa_VIe',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [19]:
es.indices.get_mapping(index="test_index")['test_index']['mappings']

{'properties': {'email_body': {'type': 'text'}}}

# 3.2 Completion

In [21]:
es.indices.delete(index="test_index", ignore_unavailable=True)
mapping={
    "properties":{
        "suggest":{ #compulsory field
            "type":"completion"
        }
    }
}
es.indices.create(index="test_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [22]:
doc1={
    "suggestion":{
        "input":["Mars", "Planet"] #copulsory field
    }
}
doc2={
    "suggestion":{
        "input":["Moon", "Satellite"] #copulsory field
    }
}

res1=es.index(index="test_index", document=doc1)
res2=es.index(index="test_index", document=doc1)

In [23]:
pprint(res1.body)
pprint(res2.body)

{'_id': 'zd9eEpUBZI6kT9oaz1K3',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}
{'_id': 'zt9eEpUBZI6kT9oaz1Lm',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}
